# Experimenting with least squares and its variants

In [1]:
%matplotlib inline

from sklearn import datasets
from scipy.optimize import fmin_bfgs
from sklearn import linear_model
import numpy as np
from numpy.linalg import norm
from numpy.linalg import inv
from sklearn.linear_model import Ridge
import math

## Data preparation

In [2]:
boston = datasets.load_boston()
data = np.array(boston.data)

The boston dataset is one of the standard regression problems used to experiment with learning algorithms. Below you can find the dataset description

In [3]:
print(boston.DESCR)

.. _boston_dataset:

Boston house prices dataset
---------------------------

**Data Set Characteristics:**  

    :Number of Instances: 506 

    :Number of Attributes: 13 numeric/categorical predictive. Median Value (attribute 14) is usually the target.

    :Attribute Information (in order):
        - CRIM     per capita crime rate by town
        - ZN       proportion of residential land zoned for lots over 25,000 sq.ft.
        - INDUS    proportion of non-retail business acres per town
        - CHAS     Charles River dummy variable (= 1 if tract bounds river; 0 otherwise)
        - NOX      nitric oxides concentration (parts per 10 million)
        - RM       average number of rooms per dwelling
        - AGE      proportion of owner-occupied units built prior to 1940
        - DIS      weighted distances to five Boston employment centres
        - RAD      index of accessibility to radial highways
        - TAX      full-value property-tax rate per $10,000
        - PTRATIO  pu

First step to apply the formulae we learnt during the lectures is to rewrite the dataset in homogeneous coordinates (i.e., we append a column of 1 to the matrix containing the examples):

In [4]:
t = np.ones(len(data)).reshape(len(data),1)
data = np.append(data, t, 1)
target = np.array(boston.target)

We now divide the data into a training set $X$ and a test set $X_\textrm{test}$.

In [5]:
X,y = data[0:400,:], target[0:400]
X_test, y_test = data[400:,:], target[400:]

# Exercise

1. Calculate the least square solution (to the regression problem outlined above) and evaluate its performances on the training set and on the test set.
1. Calculate the ridge regression solution (set lambda to 0.01) and evaluate its performances on the training set and on test set.
1. Calculate the lasso regression solution and evaluate its performances on the training set and on the test set.

## Notes

- Here it follows a list of functions you may want to use (the required packages are already imported at the beginning of this notebook) along with a very brief explanation of their purpose (`help(nomefun)` will provide you more information about function `nomefun`):
    - `transpose`: matrix transposition (e.g., `transpose(X)`)
    - `dot`: matrix multiplication (e.g., `X.dot(X2)`) 
    - `inv`: matrix inversion (e.g., `inv(X)`)
- to solve the lasso problem you will need to perform a numerical minimization of the associated loss function (as you know, a closed form solution does not exist). There are many numerical optimization algorithms available in the scipy package. My suggestion is to use `fmin_bfgs`. Here it follows an example of how to use it:
    ```python
        def f(w):
            return w[0]**2 + w[1]**2 + w[0] + w[1]
        
        w = fmin_bfgs(f, [0,0])
    ```
    note that the function may (and should) reference your data variables (i.e., $X$ and $y$).
- to evaluate the performances of your solutions use the $S$ statistic:
    $$
        S =  \sqrt{ \frac{1}{n} \sum_{i=1}^n (y_i' - y_i)^2 }
    $$
    where $y'_i$ is your model prediction for the i-th example, and $n$ is the number of examples.

In [6]:
def least_square():
    w = inv(np.transpose(X).dot(X)).dot(np.transpose(X).dot(y))
    return w
w = least_square()
print ("Pesi least square: %s" %w)

Pesi least square: [-1.91246374e-01  4.42289967e-02  5.52207977e-02  1.71631351e+00
 -1.49957220e+01  4.88773025e+00  2.60921031e-03 -1.29480799e+00
  4.84787214e-01 -1.54006673e-02 -8.08795026e-01 -1.29230427e-03
 -5.17953791e-01  2.86725996e+01]


In [7]:
Y = np.dot(X,w)
print("Statistica su dati di training",np.sqrt(np.sum((Y-y)**2)/len(Y)))
Y_test = np.dot(X_test,w)
print("Statistica su dati di test",np.sqrt(np.sum((Y_test-y_test)**2)/len(Y_test)))

Statistica su dati di training 4.722840838326382
Statistica su dati di test 6.155792280419


In [8]:
def ridge():
    lambda_ = 0.01
    temp = np.transpose(X).dot(X)
    I_x_lambda = lambda_ * np.identity(len(temp))
    temp2 = inv(temp + I_x_lambda)
    temp3 = np.dot(temp2,np.transpose(X))
    w = np.dot(temp3, y)
    return w

w_ridge = ridge()
print("Pesi Ridge",w_ridge,"\n")

Y = np.dot(X,w_ridge)
print("Statistica su dati di training",np.sqrt(np.sum((Y-y)**2)/len(Y)))

Y_test = np.dot(X_test,w_ridge)
print("Statistica su dati di test ridge",np.sqrt(np.sum((Y_test-y_test)**2)/len(Y_test)))

Pesi Ridge [-1.91440894e-01  4.42725237e-02  5.43610777e-02  1.71562849e+00
 -1.46631854e+01  4.91104161e+00  2.40041390e-03 -1.28746240e+00
  4.82736224e-01 -1.53750112e-02 -8.01618031e-01 -1.05056023e-03
 -5.16996650e-01  2.81110919e+01] 

Statistica su dati di training 4.722895265098316
Statistica su dati di test ridge 6.141787930908785


In [9]:
def f_lasso(w):
    lambda_ = 0.01
    w = (y - X.dot(w)).T.dot(y - X.dot(w)) + (lambda_ * np.linalg.norm(w,ord=1))
    return w

w_lasso = fmin_bfgs(f_lasso,np.zeros(X.shape[1]))
print("Pesi lasso: ",w_lasso,"\n")

Y = np.dot(X,w_lasso)
print("Statistica su dati di training",np.sqrt(np.sum((Y-y)**2)/len(Y)))
Y_test = np.dot(X_test,w_lasso)
print("Statistica su dati di test lasso",np.sqrt(np.sum((Y_test-y_test)**2)/len(Y_test)))

         Current function value: 8922.627021
         Iterations: 19
         Function evaluations: 577
         Gradient evaluations: 36
Pesi lasso:  [-1.91248106e-01  4.42302944e-02  5.51973360e-02  1.71613896e+00
 -1.49876514e+01  4.88814271e+00  2.60412169e-03 -1.29464530e+00
  4.84752121e-01 -1.54009081e-02 -8.08642297e-01 -1.28808181e-03
 -5.17941187e-01  2.86613795e+01] 

Statistica su dati di training 4.722840863514988
Statistica su dati di test lasso 6.155556501197483
